## 导入包

In [1]:
import numpy as np
import time
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GroupKFold, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch.optim import SGD
import torch.utils.data as Data
from torch.nn.utils import weight_norm
torch.manual_seed(0)
import math
from math import sqrt
from matplotlib import pyplot
import matplotlib.pyplot as plt
# import seaborn as sns
torch.manual_seed(0)
np.random.seed(0)


import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


## 导入数据

In [2]:
# 导入原数据
data = pd.read_csv('./ETTDataset/ETTh1.csv')
# c_all=pd.Series(data.columns)
data

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
17416,2018-06-26 16:00:00,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
17417,2018-06-26 17:00:00,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
17418,2018-06-26 18:00:00,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


In [3]:
# 导入 归一化数据
data_S = pd.read_csv('./ETTDataset/ETTh1_S.csv')
c_all=pd.Series(data_S.columns.drop('date'))
data_S

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,0.615599,0.454943,0.628980,0.467510,0.556576,0.613765,0.691018
1,2016-07-01 01:00:00,0.612708,0.459449,0.626458,0.464878,0.550279,0.620783,0.636233
2,2016-07-01 02:00:00,0.601143,0.436920,0.621438,0.459689,0.512595,0.586144,0.636233
3,2016-07-01 03:00:00,0.599698,0.450437,0.621438,0.462320,0.515693,0.599955,0.581468
4,2016-07-01 04:00:00,0.605480,0.450437,0.626458,0.467510,0.521990,0.599955,0.519656
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,0.453765,0.558574,0.458955,0.589577,0.481107,0.655196,0.299159
17416,2018-06-26 16:00:00,0.371392,0.608137,0.376064,0.599956,0.487404,0.689608,0.301955
17417,2018-06-26 17:00:00,0.550572,0.576597,0.572038,0.587018,0.506298,0.655196,0.286521
17418,2018-06-26 18:00:00,0.689299,0.576597,0.720262,0.587018,0.500000,0.634594,0.276679


In [4]:
features = ["HUFL", "HULL", "MUFL", "MULL","LUFL","LULL"]
input = data_S[features]
# 标签为 未归一化数据
target = data["OT"]

In [5]:
# # 切分训练集和测试集
split = int(input.shape[0] * 0.7)

train_input = input[:split]
test_input = input[split:]

train_target = target[:split]
test_target = target[split:]

In [6]:
# 训练集和测试集转化为张量

train_input_tensor = torch.tensor(train_input.values)
test_input_tensor = torch.tensor(test_input.values)

train_target_tensor = torch.tensor(train_target.values)
test_target_tensor = torch.tensor(test_target.values)

In [7]:
# 将input与target合并
o_data = pd.concat([input, target], axis=1)

In [8]:
# # 切分训练集和测试集
split = int(o_data.shape[0] * 0.7)
train_data = o_data[:split]
test_data = o_data[split:]

In [9]:
# 数据转化为tensor
train_data_tensor = torch.tensor(train_data.values)
test_data_tensor = torch.tensor(test_data.values)

In [ ]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        # df.iloc[:, :-1] # 选取除最后一列以外的所有列
        train_seq = input_data.iloc[:, :-1][i:i+tw]
        # 转化为numpy数据
        train_seq = train_seq.values
        train_label = input_data.iloc[:,-1][i:i+tw]
        train_label = train_label.values
        
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

In [11]:
inout_seq = []
L = len(input_data)
for i in range(L-tw):
    # df.iloc[:, :-1] # 选取除最后一列以外的所有列
    train_seq = input_data.iloc[:, :-1][i:i+tw]
    # 转化为numpy数据
    train_seq = train_seq.values
    train_label = input_data.iloc[:,-1][i:i+tw]
    train_label = train_label.values

    inout_seq.append((train_seq ,train_label))

In [31]:
# 位置编码
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__() 
        #max_len参数指定模型将能够处理的最大序列长度，而d_model参数指定输入嵌入的维数
        #pe将为输入序列中的每个位置（最多max_len）提供一行，并为每个嵌入维度提供d_model列。
        # 使用全零初始化了pe
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]
       

In [30]:
# 模型架构
class TransAm(nn.Module):
    def __init__(self,feature_size=200,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        # 初始化方法 __init__ 中，定义了几个成员变量如下：
        # model_type 是一个字符串变量，表示这个模型类型是 Transformer
        self.model_type = 'Transformer'
        # src_mask 是一个存储掩码的变量，用于遮蔽输入中的填充
        self.src_mask = None
        # pos_encoder 是一个位置编码器，用于将输入的位置信息编码到输入的向量中
        self.pos_encoder = PositionalEncoding(feature_size)
        # encoder_layer 是一个 Transformer 编码器层
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        
        # transformer_encoder 则是一个由多个编码器层构成的 Transformer 编码器
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        
        # decoder 是一个线性层，将 Transformer 编码器的输出转换为模型的输出
        # 将decoder 直接改用线性层
        self.decoder = nn.Linear(feature_size,1)
#         self.decoder = nn.Linear(1,1)
        # 初始化模型的权重和偏置
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
        # 对输入进行位置编码
        src = self.pos_encoder(src)
        # Transformer 编码器进行 编码
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        # 线性层 decoder 将编码器的输出转换为模型的输出
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask


In [29]:
# 序列
# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        # 数据和标签
        train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
        train_label = input_data[i:i+tw]
        #train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    # 转化成张量
    return torch.FloatTensor(inout_seq)

In [29]:
# 数据处理
def get_data():
#     time        = np.arange(0, 400, 0.1)
#     amplitude   = np.sin(time) + np.sin(time*0.05) +np.sin(time*0.12) *np.random.normal(-0.2, 0.2, len(time))
    
    series = pd.read_csv('./ETTDataset/daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    
    # 归一化
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
#     amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    
    # 划分训练和测试数据
    sampels = 2800
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment.. 
    # 窗口为100，将数据划分成(带标签的)训练和测试数据，并转化为张量
    # 转化训练数据
    train_sequence = create_inout_sequences(train_data,input_window)
    # 训练数据shape[2695,2,100]
    train_sequence = train_sequence[:-output_window] #todo: fix hack?

    # 转化测试数据
    #test_data = torch.FloatTensor(test_data).view(-1) 
    test_data = create_inout_sequences(test_data,input_window)
    # 训练数据shape[745,2,100]
    test_data = test_data[:-output_window] #todo: fix hack?

    return train_sequence.to(device),test_data.to(device)

In [38]:
# 数据转化：输入标签
def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]   
    # 数据切分成 输入和标签
    # 数据维度转化为[input_window,batch_size,1]
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target

In [31]:
# 训练模型
def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        # 数据切分成 输入数据和标签
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)        

        # calculate_loss_over_all_values 为True时，损失将在所有输出值和目标值上计算
        # 为False时，损失仅计算 output 和 targets 的最后 output_window 个值
        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
    
        loss.backward()
        # 执行梯度裁剪的函数,parameters参数是表示神经网络参数梯度的张量列表
        # max_norm参数是梯度的最大范数，即当参数的范数大于max_norm时，就会对梯度进行削减
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        # 计算累积的损失值
        total_loss += loss.item()
        # 计算打印日志的间隔，设置为训练集大小的5分之一:53
        log_interval = int(len(train_data) / batch_size / 5)
        # batch 是 log_interval 的倍数且不是第 0 个 batch时，开始打印日志
        if batch % log_interval == 0 and batch > 0:
            # 计算当前平均损失
            cur_loss = total_loss / log_interval
            # 计算从训练开始到现在的时间
            elapsed = time.time() - start_time
            # 打印日志，其中包括当前 epoch，batch，学习率，时间，损失值和困惑度
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            # 重置 total_loss，以便下一个 log_interval 计算新的平均损失值
            total_loss = 0
            start_time = time.time()

In [32]:
# 绘图
def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            # look like the model returns static values for the output window
            output = eval_model(data)    
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                # 最后 output_window 个数据计算loss
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            # 记录模型在测试集上的预测结果和真实值
            #test_result是一个PyTorch张量，正在与 output[-1].view(-1).cpu() 进行连接
            #[-1] 用于访问该张量的最后一个元素，view(-1)用来将 output 的最后一个元素重新形状为一个一维张量。
            #它存储了模型在测试集上的所有预测结果，每个预测结果都是一个标量。
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) #todo: check this. -> looks good to me
            # truth也是一个PyTorch张量，它存储了测试集中的所有真实标签，每个标签也是一个标量。
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
# 尝试保存结果            
    test_result = test_result.cpu().numpy()
    test_result = pd.DataFrame(test_result)
    test_result.to_csv('graph/test_result.csv',encoding='utf-8',index=False)
    len(test_result)

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
#     pyplot.plot(test_result-truth,color="green")
    # 网格线
    pyplot.grid(True, which='both')
    # 位置为y=0（即x轴）的地方添加一条水平线，线的颜色为黑色
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch%d.png'%epoch)
    # 关闭当前的图表窗口
    pyplot.close()
    
    return total_loss / i

In [33]:
# 预测下一步
# predict the next n steps based on the input data 

# def predict_future(eval_model, data_source,steps):
#     eval_model.eval() 
#     total_loss = 0.
#     test_result = torch.Tensor(0)    
#     truth = torch.Tensor(0)
#     _ , data = get_batch(data_source, 0,1)
#     with torch.no_grad():
#         for i in range(0, steps,1):
#             input = torch.clone(data[-input_window:])
#             input[-output_window:] = 0 
#             output = eval_model(data[-input_window:])                        
#             data = torch.cat((data, output[-1:]))
            
#     data = data.cpu().view(-1)
    
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)

    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png'%steps)
    pyplot.close()

In [34]:
# 评估
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            if calculate_loss_over_all_values:
                total_loss += len(data[0])* criterion(output, targets).cpu().item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()            
    return total_loss / len(data_source)

In [35]:
# 超参数
input_window = 100
output_window = 5
batch_size = 10 # batch size

# calculate_loss_over_all_values = False
calculate_loss_over_all_values = True

lr = 0.007

# 初始化为正无穷大，确保第一次比较时，任何值都比 best_val_loss 更小
best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

In [36]:
# 加载数据和 模型
train_data, val_data = get_data()
model = TransAm().to(device)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5156\4005502798.py:6: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  series = pd.read_csv('./ETTDataset/daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)


In [37]:
# 查看model
model

TransAm(
  (pos_encoder): PositionalEncoding()
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
    )
    (linear1): Linear(in_features=200, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=200, bias=True)
    (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=2048, bias=True)
        (dropout): Dropout

In [38]:
# 模型训练

criterion = nn.MSELoss()

#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# 学习率调度器
# 每一个epoch都会将学习率乘以0.98,
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)


for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    
    if(epoch % 10 == 0):
        # 每10个epoch开始绘图
        val_loss = plot_and_loss(model, val_data,epoch)
        # 200数值代表预测未来的步数
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)
        
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        # 将 best_val_loss 更新为当前的验证损失值
        best_val_loss = val_loss
        best_model = model

    scheduler.step() 

D:\Anaconda\envs\xai3\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |    53/  269 batches | lr 0.007000 | 14.61 ms | loss 3.48887 | ppl    32.75
| epoch   1 |   106/  269 batches | lr 0.007000 | 10.45 ms | loss 0.04615 | ppl     1.05
| epoch   1 |   159/  269 batches | lr 0.007000 | 10.45 ms | loss 0.02783 | ppl     1.03
| epoch   1 |   212/  269 batches | lr 0.007000 | 10.39 ms | loss 0.01752 | ppl     1.02
| epoch   1 |   265/  269 batches | lr 0.007000 | 10.32 ms | loss 0.01507 | ppl     1.02
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  3.19s | valid loss 0.01674 | valid ppl     1.02
-----------------------------------------------------------------------------------------
| epoch   2 |    53/  269 batches | lr 0.006723 | 10.62 ms | loss 0.01732 | ppl     1.02
| epoch   2 |   106/  269 batches | lr 0.006723 | 10.40 ms | loss 0.01497 | ppl     1.02
| epoch   2 |   159/  269 batches | lr 0.006723 | 10.35 ms | loss 0.01696 | ppl     1.02
| epoch   2 |   212/  269 batche

| epoch  13 |    53/  269 batches | lr 0.005383 | 10.86 ms | loss 0.00855 | ppl     1.01
| epoch  13 |   106/  269 batches | lr 0.005383 | 10.23 ms | loss 0.00702 | ppl     1.01
| epoch  13 |   159/  269 batches | lr 0.005383 | 10.46 ms | loss 0.00620 | ppl     1.01
| epoch  13 |   212/  269 batches | lr 0.005383 | 10.27 ms | loss 0.00503 | ppl     1.01
| epoch  13 |   265/  269 batches | lr 0.005383 | 10.75 ms | loss 0.00546 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  13 | time:  3.00s | valid loss 0.00550 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  14 |    53/  269 batches | lr 0.005275 | 10.83 ms | loss 0.00776 | ppl     1.01
| epoch  14 |   106/  269 batches | lr 0.005275 | 10.45 ms | loss 0.00680 | ppl     1.01
| epoch  14 |   159/  269 batches | lr 0.005275 | 10.43 ms | loss 0.00643 | ppl     1.01
| epoch  14 |   212/  269 batche

| epoch  25 |    53/  269 batches | lr 0.004224 | 10.77 ms | loss 0.00563 | ppl     1.01
| epoch  25 |   106/  269 batches | lr 0.004224 | 10.89 ms | loss 0.00514 | ppl     1.01
| epoch  25 |   159/  269 batches | lr 0.004224 | 10.62 ms | loss 0.00459 | ppl     1.00
| epoch  25 |   212/  269 batches | lr 0.004224 | 10.43 ms | loss 0.00416 | ppl     1.00
| epoch  25 |   265/  269 batches | lr 0.004224 | 10.41 ms | loss 0.00440 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  25 | time:  3.02s | valid loss 0.00465 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  26 |    53/  269 batches | lr 0.004140 | 10.51 ms | loss 0.00548 | ppl     1.01
| epoch  26 |   106/  269 batches | lr 0.004140 | 10.34 ms | loss 0.00501 | ppl     1.01
| epoch  26 |   159/  269 batches | lr 0.004140 | 10.30 ms | loss 0.00469 | ppl     1.00
| epoch  26 |   212/  269 batche

| epoch  37 |    53/  269 batches | lr 0.003315 | 10.87 ms | loss 0.00492 | ppl     1.00
| epoch  37 |   106/  269 batches | lr 0.003315 | 10.67 ms | loss 0.00458 | ppl     1.00
| epoch  37 |   159/  269 batches | lr 0.003315 | 10.81 ms | loss 0.00420 | ppl     1.00
| epoch  37 |   212/  269 batches | lr 0.003315 | 10.97 ms | loss 0.00348 | ppl     1.00
| epoch  37 |   265/  269 batches | lr 0.003315 | 10.35 ms | loss 0.00348 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  37 | time:  3.05s | valid loss 0.00425 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  38 |    53/  269 batches | lr 0.003249 | 10.94 ms | loss 0.00470 | ppl     1.00
| epoch  38 |   106/  269 batches | lr 0.003249 | 10.91 ms | loss 0.00415 | ppl     1.00
| epoch  38 |   159/  269 batches | lr 0.003249 | 11.11 ms | loss 0.00407 | ppl     1.00
| epoch  38 |   212/  269 batche

| epoch  49 |    53/  269 batches | lr 0.002601 | 10.92 ms | loss 0.00388 | ppl     1.00
| epoch  49 |   106/  269 batches | lr 0.002601 | 11.17 ms | loss 0.00339 | ppl     1.00
| epoch  49 |   159/  269 batches | lr 0.002601 | 11.04 ms | loss 0.00340 | ppl     1.00
| epoch  49 |   212/  269 batches | lr 0.002601 | 11.26 ms | loss 0.00321 | ppl     1.00
| epoch  49 |   265/  269 batches | lr 0.002601 | 10.70 ms | loss 0.00327 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  49 | time:  3.13s | valid loss 0.00380 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  50 |    53/  269 batches | lr 0.002549 | 11.68 ms | loss 0.00397 | ppl     1.00
| epoch  50 |   106/  269 batches | lr 0.002549 | 10.89 ms | loss 0.00369 | ppl     1.00
| epoch  50 |   159/  269 batches | lr 0.002549 | 10.66 ms | loss 0.00367 | ppl     1.00
| epoch  50 |   212/  269 batche

| epoch  61 |    53/  269 batches | lr 0.002041 | 10.53 ms | loss 0.00335 | ppl     1.00
| epoch  61 |   106/  269 batches | lr 0.002041 | 10.45 ms | loss 0.00320 | ppl     1.00
| epoch  61 |   159/  269 batches | lr 0.002041 | 10.41 ms | loss 0.00305 | ppl     1.00
| epoch  61 |   212/  269 batches | lr 0.002041 | 10.88 ms | loss 0.00250 | ppl     1.00
| epoch  61 |   265/  269 batches | lr 0.002041 | 10.95 ms | loss 0.00274 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  61 | time:  3.03s | valid loss 0.00334 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  62 |    53/  269 batches | lr 0.002000 | 10.73 ms | loss 0.00340 | ppl     1.00
| epoch  62 |   106/  269 batches | lr 0.002000 | 10.30 ms | loss 0.00318 | ppl     1.00
| epoch  62 |   159/  269 batches | lr 0.002000 | 10.24 ms | loss 0.00314 | ppl     1.00
| epoch  62 |   212/  269 batche

| epoch  73 |    53/  269 batches | lr 0.001602 | 11.12 ms | loss 0.00329 | ppl     1.00
| epoch  73 |   106/  269 batches | lr 0.001602 | 10.94 ms | loss 0.00286 | ppl     1.00
| epoch  73 |   159/  269 batches | lr 0.001602 | 10.77 ms | loss 0.00288 | ppl     1.00
| epoch  73 |   212/  269 batches | lr 0.001602 | 10.79 ms | loss 0.00250 | ppl     1.00
| epoch  73 |   265/  269 batches | lr 0.001602 | 10.39 ms | loss 0.00272 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  73 | time:  3.07s | valid loss 0.00354 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  74 |    53/  269 batches | lr 0.001570 | 11.21 ms | loss 0.00351 | ppl     1.00
| epoch  74 |   106/  269 batches | lr 0.001570 | 10.58 ms | loss 0.00290 | ppl     1.00
| epoch  74 |   159/  269 batches | lr 0.001570 | 10.50 ms | loss 0.00279 | ppl     1.00
| epoch  74 |   212/  269 batche

| epoch  85 |    53/  269 batches | lr 0.001257 | 10.54 ms | loss 0.00314 | ppl     1.00
| epoch  85 |   106/  269 batches | lr 0.001257 | 10.49 ms | loss 0.00277 | ppl     1.00
| epoch  85 |   159/  269 batches | lr 0.001257 | 10.58 ms | loss 0.00274 | ppl     1.00
| epoch  85 |   212/  269 batches | lr 0.001257 | 10.57 ms | loss 0.00236 | ppl     1.00
| epoch  85 |   265/  269 batches | lr 0.001257 | 10.59 ms | loss 0.00253 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  85 | time:  3.00s | valid loss 0.00328 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  86 |    53/  269 batches | lr 0.001232 | 10.82 ms | loss 0.00315 | ppl     1.00
| epoch  86 |   106/  269 batches | lr 0.001232 | 10.37 ms | loss 0.00283 | ppl     1.00
| epoch  86 |   159/  269 batches | lr 0.001232 | 10.40 ms | loss 0.00272 | ppl     1.00
| epoch  86 |   212/  269 batche

| epoch  97 |    53/  269 batches | lr 0.000986 | 10.62 ms | loss 0.00295 | ppl     1.00
| epoch  97 |   106/  269 batches | lr 0.000986 | 10.44 ms | loss 0.00252 | ppl     1.00
| epoch  97 |   159/  269 batches | lr 0.000986 | 10.51 ms | loss 0.00251 | ppl     1.00
| epoch  97 |   212/  269 batches | lr 0.000986 | 10.81 ms | loss 0.00215 | ppl     1.00
| epoch  97 |   265/  269 batches | lr 0.000986 | 10.62 ms | loss 0.00233 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  97 | time:  3.02s | valid loss 0.00288 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  98 |    53/  269 batches | lr 0.000967 | 10.90 ms | loss 0.00286 | ppl     1.00
| epoch  98 |   106/  269 batches | lr 0.000967 | 10.48 ms | loss 0.00249 | ppl     1.00
| epoch  98 |   159/  269 batches | lr 0.000967 | 10.32 ms | loss 0.00252 | ppl     1.00
| epoch  98 |   212/  269 batche

In [39]:
best_val_loss

0.002758366634581713

In [43]:
best_model

TransAm(
  (pos_encoder): PositionalEncoding()
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
    )
    (linear1): Linear(in_features=200, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=200, bias=True)
    (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=2048, bias=True)
        (dropout): Dropout

In [45]:
# 尝试保存模型 参数
output_model_dir = './models/'
#如果一开始用了并行训练最好加上这句
model_to_save = best_model.module if hasattr(model, 'module') else model
#这样保存的是模型参数，记得格式是.pt
torch.save(model_to_save.state_dict(),output_model_dir+"TRMmodel.pt")


# # 加载模型
# #因为是自定义模型呀
# model = Model()
# #拿到保存的参数
# model_static_dict = torch.load(output_model_dir+"TRMmodel.pt")
# #把参数加载到模型中
# model.load_state_dict(model_static_dict)

In [47]:
# 尝试保存模型
torch.save(model, output_model_dir+"TRM1model.pt")
# 加载模型
model = torch.load(output_model_dir+"TRM1model.pt")

In [48]:
model

TransAm(
  (pos_encoder): PositionalEncoding()
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
    )
    (linear1): Linear(in_features=200, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=200, bias=True)
    (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=2048, bias=True)
        (dropout): Dropout